In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")

# 현재 기동된 스파크 애플리케이션의 포트를 확인하기 위해 스파크 정보를 출력합니다
spark

In [8]:
# 스트림 테이블을 주기적으로 조회하는 함수 (name: 이름, sql: Spark SQL, iterations: 반복횟수, sleep_secs: 인터벌)
def displayStream(name, sql, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)              # 출력 Cell 을 지웁니다
        display('[' + name + '] Iteration: '+str(i)+', Query: '+sql)
        display(spark.sql(sql))              # Spark SQL 을 수행합니다
        sleep(sleep_secs)                    # sleep_secs 초 만큼 대기합니다
        i += 1

# 스트림 쿼리의 상태를 주기적으로 조회하는 함수 (name: 이름, query: Streaming Query, iterations: 반복횟수, sleep_secs: 인터벌)
def displayStatus(name, query, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)      # Output Cell 의 내용을 지웁니다
        display('[' + name + '] Iteration: '+str(i)+', Status: '+query.status['message'])
        display(query.lastProgress)  # 마지막 수행된 쿼리의 상태를 출력합니다
        sleep(sleep_secs)            # 지정된 시간(초)을 대기합니다
        i += 1

In [9]:
source_topic = "events"
source_offset = "latest"
query_name = "kafka_sink"
sink_topic = "events_stage"

kafkaReader = (
    spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:9093")
  .option("subscribe", source_topic)
  .option("startingOffsets", source_offset)
  .load()
)
kafkaReader.printSchema()

kafkaSchema = (
    StructType()
    .add(StructField("id", LongType()))
    .add(StructField("hello", StringType()))
    .add(StructField("time", StringType()))
)

kafkaSelector = (
    kafkaReader
    .select(
        col("key").cast("string"),
        from_json(col("value").cast("string"), kafkaSchema).alias("events")
    )
    .selectExpr("events.id % 10 as mod_id", "events.*")
)

kafkaSelector.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

root
 |-- mod_id: long (nullable = true)
 |-- id: long (nullable = true)
 |-- hello: string (nullable = true)
 |-- time: string (nullable = true)



In [10]:
namePath = f"{work_dir}/data/names"
nameStatic = (
    spark
    .read
    .option("inferSchema", "true")
    .json(namePath)
)
nameStatic.printSchema()
nameStatic.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- uid: long (nullable = true)

+-----+---+
|name |uid|
+-----+---+
|zero |0  |
|one  |1  |
|two  |2  |
|three|3  |
|four |4  |
|five |5  |
|six  |6  |
|seven|7  |
|eight|8  |
|nine |9  |
+-----+---+



In [11]:
joinExpression = (kafkaSelector.mod_id == nameStatic.uid)
kappaSelector = kafkaSelector.join(nameStatic, joinExpression, "leftOuter")

In [ ]:
# query_name = "memorySink"
# staticWriter = (
#     kappaSelector
#     .selectExpr("time", "id as user_id", "name as user_name", "hello", "uid")
#     .selectExpr("user_id as key", "to_json(struct(*)) as value")
#     .writeStream
#     .queryName(query_name)
#     .format("memory")
#     .outputMode("append")
# )

# checkpointLocation = f"{work_dir}/tmp/{query_name}"
# !rm -rf $checkpointLocation

# staticTrigger = (
#     staticWriter
#     .trigger(processingTime="5 second")
#     .option("checkpointLocation", checkpointLocation)
# )

In [ ]:
# staticQuery = staticTrigger.start()
# displayStream(query_name, f"select * from {query_name} order by key desc", 40, 5)
# staticQuery.explain(True)
# staticQuery.stop()

In [ ]:
# !rm -rf $checkpointLocation
# staticQuery.stop()

In [13]:
staticWriter = (
    kappaSelector
    .selectExpr("time", "id as user_id", "name as user_name", "hello", "uid")
    .selectExpr("cast(user_id as string) as key", "to_json(struct(*)) as value")
    .writeStream
    .queryName(query_name)
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9093")
    .option("topic", sink_topic)
    .outputMode("append")
)

checkpointLocation = f"{work_dir}/tmp/{query_name}"
!rm -rf $checkpointLocation

staticTrigger = (
    staticWriter
    .trigger(processingTime="5 second")
    .option("checkpointLocation", checkpointLocation)
)

In [15]:
staticQuery = staticTrigger.start()
displayStatus(query_name, staticQuery, 1000, 6)
staticQuery.stop()

'[kafkaSink] Iteration: 1000, Status: Processing new data'

{'id': '7ae7ea73-13f6-4b9b-9ffb-450c85c2f465',
 'runId': '6687ebf7-5590-4825-9225-a5129f9e12d2',
 'name': 'kafkaSink',
 'timestamp': '2022-10-01T00:35:55.001Z',
 'batchId': 1319,
 'numInputRows': 4,
 'inputRowsPerSecond': 0.7998400319936012,
 'processedRowsPerSecond': 5.813953488372094,
 'durationMs': {'addBatch': 570,
  'getBatch': 1,
  'latestOffset': 1,
  'queryPlanning': 5,
  'triggerExecution': 688,
  'walCommit': 65},
 'stateOperators': [],
 'sources': [{'description': 'KafkaV2[Subscribe[events]]',
   'startOffset': {'events': {'0': 7215}},
   'endOffset': {'events': {'0': 7219}},
   'latestOffset': {'events': {'0': 7219}},
   'numInputRows': 4,
   'inputRowsPerSecond': 0.7998400319936012,
   'processedRowsPerSecond': 5.813953488372094,
   'metrics': {'avgOffsetsBehindLatest': '0.0',
    'maxOffsetsBehindLatest': '0',
    'minOffsetsBehindLatest': '0'}}],
 'sink': {'description': 'org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@28c98734',
  'numOutputRows': 4}}